# Klasifikasi Berita Menggunakan Feature Ekstrakstion Bag Of Word (BOW) dan TF-IDF dengan Metode SVM dan Random Forest

## 1. Import Library/Modul



In [ ]:
# Impor library yang diperlukan
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True



**1. Import Library/Modul**

Saya mengimpor berbagai library (modul) yang diperlukan untuk melakukan analisis teks dan pemodelan klasifikasi. Berikut beberapa modul yang telah diimpor:

- `pandas`: Modul ini digunakan untuk membaca, memanipulasi, dan menganalisis data dalam bentuk tabel (DataFrames).

- `numpy`: Modul ini umumnya digunakan untuk operasi matematika dan manipulasi array, yang sering diperlukan dalam analisis data.

- `CountVectorizer` dan `TfidfVectorizer`: Modul ini digunakan untuk mengubah teks menjadi vektor numerik. `CountVectorizer` menghitung frekuensi munculnya kata dalam dokumen, sedangkan `TfidfVectorizer` mengukur pentingnya kata dalam suatu dokumen dalam konteks koleksi dokumen.

- `train_test_split`: Modul ini digunakan untuk membagi dataset menjadi data pelatihan dan data pengujian.

- `SVC` (Support Vector Classifier) dan `RandomForestClassifier`: Modul ini digunakan untuk melatih model klasifikasi. `SVC` adalah model klasifikasi berdasarkan Support Vector Machines, sementara `RandomForestClassifier` adalah model klasifikasi berdasarkan algoritma Random Forest.

- `accuracy_score`, `classification_report`, dan `confusion_matrix`: Modul ini digunakan untuk mengevaluasi performa model klasifikasi, termasuk menghitung akurasi, presisi, recall, F1-score, dan matriks kebingungan.

- `LabelEncoder`: Modul ini digunakan untuk mengkodekan label kategori ke dalam bentuk numerik.

- `stopwords` dari `nltk.corpus`: Modul ini berisi daftar kata-kata umum yang sering diabaikan dalam analisis teks karena mereka tidak memiliki makna khusus (contoh: "dan", "atau", "di").

- `re`: Modul ini digunakan untuk bekerja dengan ekspresi reguler, yang berguna untuk membersihkan teks.

- `nltk`: Natural Language Toolkit, adalah perpustakaan untuk pemrosesan bahasa alami, yang mencakup berbagai alat dan sumber daya untuk analisis teks.

Dengan mengimpor modul-modul ini, saya telah mempersiapkan alat yang saya butuhkan untuk melakukan pemrosesan teks dan pemodelan klasifikasi dalam kode berikutnya.

## 2. Import Dataset

Langkah berikutnya adalah saya mengimpor dataset yang saya perlukan dalam proyek ini. Dataset ini saya kumpulkan secara manual dengan mengunjungi berbagai sumber berita seperti TVOne, Kompas TV, dan lainnya. Setelah itu, saya melakukan labeling manual dengan menggunakan tiga kategori, yaitu "Politik," "Kesehatan," dan "Olahraga." Totalnya, saya mengumpulkan 12 berita dari berbagai sumber yang berbeda.

Dalam tahap ini, saya mengumpulkan data langsung dari sumber aslinya, seperti situs web berita atau sumber informasi lainnya. Saya melakukan pengumpulan data ini untuk digunakan dalam proyek analisis teks selanjutnya.

In [ ]:
# Impor data dari file Excel
data = pd.read_excel("news_dataset.xlsx", index_col='No')
data

,Teks,Media,Label
No,,,
1,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...,@tvOneNews,Politik
2,"Anggota Komite Eksekutif (Exco) PSSI, Arya Sin...",@headlinebogor,Olahraga
3,Polri mengerahkan 13.246 personel untuk memast...,@inewsdotid,Olahraga
4,Rakhmatullaev menilai timnas Indonesia U-17 di...,@GOAL_ID,Olahraga
5,Kasus cacar monyet atau yang belakangan resmi ...,@detikHealth,Kesehatan
6,Dermatolog Hanny Nilasari mengimbau orang yang...,@tempodotco,Kesehatan
7,Ketua MK Anwar Usman dinilai terbukti telah me...,@NarasiNewsroom,Politik
8,Hasil survei terbaru lembaga kredibel Charta ...,@tvOneNews,Politik
9,Satu Orang Warga Bandung Positif Cacar Monyet....,@KompasTV,Kesehatan



Selanjutnya, saya mengimpor dataset yang telah saya siapkan sebelumnya dari file Excel yang disebut "news_dataset.xlsx." Saya menggunakan perpustakaan Pandas dalam Python untuk melakukan ini. Dataset ini memiliki kolom-kolom seperti "Teks," "Media," "Label," dan "clean_text," yang masing-masing memiliki makna sebagai berikut:

- **Teks**: Ini adalah isi berita yang saya kumpulkan dari berbagai sumber. Teks ini akan menjadi fokus utama analisis teks yang akan saya lakukan dalam proyek ini.

- **Media**: Kolom ini mencantumkan sumber berita atau media dari mana saya mengambil setiap berita. Ini penting untuk mengetahui asal-usul berita dan untuk analisis selanjutnya.

- **Label**: Kolom ini berisi label atau kategori yang saya berikan untuk setiap berita. Saya telah memberikan tiga label: "Politik," "Kesehatan," dan "Olahraga." Label ini akan digunakan sebagai dasar untuk analisis sentimen dan klasifikasi teks.



## 3. Preprocessing Data

Pada tahap ini, saya akan menjelaskan bagaimana saya melakukan preprocessing atau pemrosesan awal terhadap data teks yang telah saya impor sebelumnya. Tahap ini sangat penting dalam analisis teks karena membantu untuk membersihkan dan mempersiapkan data sebelum dilakukan analisis sentimen dan klasifikasi.

In [ ]:
# Pembersihan Teks
def clean_text(text):
    # Mengubah teks ke huruf kecil
    text = text.lower()
    # Menghapus simbol dan karakter non-alfanumerik
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Menghapus kata-kata stopwords
    stop_words = set(stopwords.words('indonesian'))
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])
    return text

data['clean_text'] = data['Teks'].apply(clean_text)
data

,Teks,Media,Label,clean_text
No,,,,
1,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...,@tvOneNews,Politik,majelis kehormatan mahkamah konstitusi mkmk me...
2,"Anggota Komite Eksekutif (Exco) PSSI, Arya Sin...",@headlinebogor,Olahraga,anggota komite eksekutif exco pssi arya sinuli...
3,Polri mengerahkan 13.246 personel untuk memast...,@inewsdotid,Olahraga,polri mengerahkan 13246 personel piala dunia u...
4,Rakhmatullaev menilai timnas Indonesia U-17 di...,@GOAL_ID,Olahraga,rakhmatullaev menilai timnas indonesia u17 dih...
5,Kasus cacar monyet atau yang belakangan resmi ...,@detikHealth,Kesehatan,cacar monyet resmi dinamai mpox meningkat indo...
6,Dermatolog Hanny Nilasari mengimbau orang yang...,@tempodotco,Kesehatan,dermatolog hanny nilasari mengimbau orang meng...
7,Ketua MK Anwar Usman dinilai terbukti telah me...,@NarasiNewsroom,Politik,ketua mk anwar usman dinilai terbukti pelangga...
8,Hasil survei terbaru lembaga kredibel Charta ...,@tvOneNews,Politik,hasil survei terbaru lembaga kredibel charta p...
9,Satu Orang Warga Bandung Positif Cacar Monyet....,@KompasTV,Kesehatan,orang warga bandung positif cacar monyet pasie...



Pada tahap ini, saya akan menjelaskan kode yang digunakan untuk membersihkan teks dalam dataset. Tujuan dari tahap ini adalah untuk membersihkan teks dari elemen-elemen yang tidak relevan atau mengganggu sehingga teks yang dihasilkan lebih sesuai untuk analisis sentimen dan klasifikasi.

Berikut adalah penjelasan tahap demi tahap kode yang digunakan:

**1. Mengubah Teks Menjadi Huruf Kecil**
   - Pada tahap pertama, kode ini mengubah seluruh teks dalam dataset menjadi huruf kecil. Ini penting untuk memastikan konsistensi dalam teks. Misalnya, "Kata" dan "kata" dianggap sama dalam analisis.

**2. Menghapus Simbol dan Karakter Non-Alfanumerik**
   - Selanjutnya, kode ini membersihkan teks dari simbol dan karakter non-alfanumerik. Hal ini dilakukan dengan menggunakan ekspresi reguler (regular expression) untuk menghilangkan karakter yang tidak termasuk huruf atau angka. Simbol dan karakter non-alfanumerik seperti tanda baca dan karakter khusus dianggap tidak relevan dalam analisis teks.

**3. Menghapus Kata-kata Stopwords**
   - Tahap selanjutnya adalah menghapus kata-kata stopwords dari teks. Stopwords adalah kata-kata umum dalam bahasa yang sering muncul dalam teks tetapi cenderung tidak memiliki makna yang kuat dalam konteks analisis. Penghapusan kata-kata stopwords membantu dalam fokus pada kata-kata kunci yang lebih penting dalam analisis.

**4. Teks Hasil Pembersihan**
   - Setelah menjalankan kode pembersihan, hasil dari tahap ini adalah dataset yang memiliki kolom baru yang berisi teks yang telah dibersihkan. Teks ini lebih sesuai untuk analisis sentimen dan klasifikasi.

Tahap ini adalah salah satu langkah penting dalam persiapan data sebelum melakukan analisis teks. Dengan melakukan pembersihan, kita memastikan bahwa data yang digunakan dalam analisis lebih berkualitas dan relevan.

## 4. Feature Extraction/ Text Representation

Feature extraction atau representasi teks adalah proses mengubah teks menjadi bentuk yang dapat digunakan oleh algoritma pembelajaran mesin. Dalam konteks analisis teks, tujuannya adalah mengubah teks menjadi vektor (kumpulan angka) sehingga model pembelajaran mesin dapat memahami dan memprosesnya. Representasi teks adalah salah satu langkah kunci dalam analisis teks dan pemrosesan bahasa alami. Dalam proses implementasi nya saya menggunakan metode Bag Of Words dan TF-IDF.

### 1. Metode Bag of Words (BoW)

Metode Bag of Words (BoW) adalah salah satu metode umum yang digunakan dalam representasi teks. Ini bekerja dengan cara menganggap setiap dokumen atau kalimat sebagai kumpulan kata atau token, dan menghitung berapa kali setiap kata muncul dalam dokumen tersebut. Hasilnya adalah vektor di mana setiap elemen mewakili frekuensi kata dalam dokumen. Keunggulan dari BoW adalah sederhana dan efektif, tetapi itu tidak memperhitungkan urutan kata dalam teks.


In [ ]:
# Text Representation Menggunakan CountVectorizer
vectorizer = CountVectorizer(max_features=50, min_df=3)
X_count = vectorizer.fit_transform(data['clean_text'])


Code ini bertujuan untuk menerapkan metode **CountVectorizer** pada teks yang telah dibersihkan sebelumnya. CountVectorizer adalah salah satu teknik representasi teks yang digunakan untuk mengubah teks menjadi vektor dengan mempertimbangkan frekuensi kata-kata dalam dokumen.

**Langkah-langkah yang dilakukan:**

1. **Inisialisasi CountVectorizer**: Pertama, kita membuat objek CountVectorizer. Dalam contoh ini, kita membatasi jumlah fitur (kata) yang dihasilkan menjadi 50 dengan menggunakan parameter `max_features=50`. Ini berarti hanya 50 kata dengan frekuensi tertinggi akan digunakan dalam vektorisasi. Selain itu, kita juga mengatur `min_df=3`, yang berarti hanya kata-kata yang muncul setidaknya 3 kali dalam seluruh dataset akan diikutkan dalam representasi.

2. **Vektorisasi Teks**: Kemudian, kita menggunakan objek CountVectorizer yang telah dibuat untuk mengonversi teks dalam kolom "clean_text" dari DataFrame `data` menjadi vektor. Hasil vektorisasi disimpan dalam variabel `X_count`.

Hasil dari langkah ini adalah kita memiliki representasi vektor dari teks yang dapat digunakan sebagai input untuk model pembelajaran mesin. Vektor tersebut akan memiliki panjang 50 (sesuai dengan `max_features` yang telah diatur) dan setiap elemen vektor akan merepresentasikan frekuensi kata dalam teks yang cocok dengan fitur-fitur yang telah ditentukan. Representasi ini akan membantu dalam melatih dan menguji model pembelajaran mesin.

In [ ]:
# Konversi hasil vektorisasi menjadi DataFrame
df1 = pd.DataFrame(X_count.toarray(), columns=vectorizer.get_feature_names_out(), index=data.index)
df1

,2023,anggota,anwar,bandung,cacar,dunia,hasil,indonesia,infeksi,jabatan,...,piala,potensi,presiden,rumah,surabaya,terkait,u17,usia,usman,virus
No,,,,,,,,,,,,,,,,,,,,,
1,0,1,3,0,0,0,2,0,0,1,...,0,1,2,0,0,0,0,0,2,0
2,1,1,0,1,0,2,0,0,0,0,...,2,0,0,0,1,0,2,0,0,0
3,2,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,2,0,0,0
4,2,0,0,0,0,2,0,3,0,0,...,2,0,0,1,0,0,6,0,0,0
5,1,0,0,0,2,0,0,3,0,0,...,0,1,0,0,0,0,0,0,0,0
6,1,0,0,0,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
7,0,1,9,0,0,0,1,0,0,2,...,0,1,0,0,0,2,0,2,4,0
8,0,0,1,0,0,0,1,0,0,0,...,0,0,2,0,1,0,0,0,1,0
9,0,0,0,3,2,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1


Code ini bertujuan untuk mengonversi hasil vektorisasi yang telah dilakukan dengan CountVectorizer menjadi sebuah DataFrame. DataFrame adalah struktur data tabel yang memungkinkan kita untuk dengan mudah melihat, memanipulasi, dan menganalisis data.

**Langkah-langkah yang dilakukan:**

1. **Konversi menjadi DataFrame**: Kita menggunakan `pd.DataFrame()` untuk mengubah matriks hasil vektorisasi (X_count) menjadi sebuah DataFrame. Dalam konstruksi DataFrame, kita menyediakan tiga argumen:
   - `X_count.toarray()`: Ini adalah matriks hasil vektorisasi yang dikonversi menjadi array NumPy.
   - `columns=vectorizer.get_feature_names_out()`: Ini adalah daftar kata-kata yang digunakan sebagai fitur dalam vektorisasi. Fungsi `get_feature_names_out()` digunakan untuk mendapatkan daftar kata-kata ini.
   - `index=data.index`: Ini adalah indeks untuk DataFrame, yang kita ambil dari indeks data awal, sehingga kita dapat mengaitkan setiap baris DataFrame dengan entri aslinya di dataset.

Hasil dari langkah ini adalah sebuah DataFrame yang berisi representasi vektor dari teks dalam dataset kita. Setiap kolom dalam DataFrame ini akan mewakili satu kata atau fitur dalam representasi vektor teks, dan setiap baris akan mewakili satu sampel data atau dokumen dalam dataset. Dengan DataFrame ini, kita dapat dengan mudah melakukan analisis lanjutan, seperti melihat kata-kata mana yang paling penting dalam membedakan kategori dalam dataset.

### 3. Term Frequency-Inverse Document Frequency (TF-IDF)

TF-IDF adalah metode lain untuk menggambarkan teks yang menghitung pentingnya kata dalam dokumen dengan memperhatikan seberapa sering kata itu muncul dalam dokumen dan seberapa umum kata itu dalam seluruh koleksi dokumen. Ini memberikan bobot yang lebih tinggi pada kata-kata yang muncul jarang dalam dokumen tetapi mungkin penting dalam membedakan dokumen satu sama lain.

In [ ]:
# Text Representation Menggunakan TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=50, min_df=3)
X_tfidf = tfidf_vectorizer.fit_transform(data['clean_text'])

Code di atas bertujuan untuk menerapkan representasi teks menggunakan metode "Term Frequency-Inverse Document Frequency" (TF-IDF) dengan bantuan `TfidfVectorizer`.

**Langkah-langkah yang dilakukan:**

1. **Inisialisasi TF-IDF Vectorizer**: Pertama-tama, kita membuat objek `tfidf_vectorizer` dengan menginisialisasi `TfidfVectorizer`. Ini adalah alat yang digunakan untuk mengubah teks menjadi representasi TF-IDF. Parameter yang diberikan termasuk `max_features` dan `min_df`.
   - `max_features`: Ini mengatur jumlah maksimum fitur (kata-kata) yang akan disimpan dalam vektor TF-IDF. Dalam hal ini, maksimum 50 kata akan disimpan.
   - `min_df`: Ini adalah frekuensi kata minimum yang diperlukan agar kata tersebut disertakan dalam vektor TF-IDF. Dalam hal ini, hanya kata-kata yang muncul minimal 3 kali dalam dataset akan disertakan.

2. **Vektorisasi TF-IDF**: Selanjutnya, kita menggunakan `tfidf_vectorizer` yang telah diinisialisasi untuk mengonversi teks dalam kolom 'clean_text' dari dataframe `data` menjadi representasi vektor TF-IDF. Hasil vektorisasi disimpan dalam `X_tfidf`.

Hasil dari langkah ini adalah matriks yang menggambarkan teks dalam bentuk representasi TF-IDF. Setiap baris mewakili satu dokumen atau sampel teks dalam dataset, sedangkan setiap kolom mewakili kata-kata atau fitur dalam bentuk TF-IDF. Dengan representasi ini, kita dapat menganalisis teks secara lebih mendalam, mengukur pentingnya kata-kata dalam dokumen, atau menggunakannya sebagai input untuk model pembelajaran mesin.

In [ ]:
# Konversi hasil vektorisasi menjadi DataFrame
df2 = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=data.index)
df2

,2023,anggota,anwar,bandung,cacar,dunia,hasil,indonesia,infeksi,jabatan,...,piala,potensi,presiden,rumah,surabaya,terkait,u17,usia,usman,virus
No,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.158286,0.474857,0.000000,0.000000,0.000000,0.316571,0.000000,0.000000,0.158286,...,0.000000,0.158286,0.316571,0.000000,0.000000,0.000000,0.000000,0.000000,0.316571,0.000000
2,0.157651,0.212143,0.000000,0.212143,0.000000,0.380829,0.000000,0.000000,0.000000,0.000000,...,0.380829,0.000000,0.000000,0.000000,0.212143,0.000000,0.380829,0.000000,0.000000,0.000000
3,0.560046,0.000000,0.000000,0.000000,0.000000,0.338218,0.000000,0.000000,0.000000,0.000000,...,0.338218,0.000000,0.000000,0.000000,0.000000,0.000000,0.676436,0.000000,0.000000,0.000000
4,0.217007,0.000000,0.000000,0.000000,0.000000,0.262106,0.000000,0.393159,0.000000,0.000000,...,0.262106,0.000000,0.000000,0.146008,0.000000,0.000000,0.786319,0.000000,0.000000,0.000000
5,0.186305,0.000000,0.000000,0.000000,0.450047,0.000000,0.000000,0.675071,0.000000,0.000000,...,0.000000,0.250701,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.199819,0.000000,0.000000,0.000000,0.724039,0.000000,0.000000,0.000000,0.268886,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.268886
7,0.000000,0.084661,0.761947,0.000000,0.000000,0.000000,0.084661,0.000000,0.000000,0.169321,...,0.000000,0.084661,0.000000,0.000000,0.000000,0.151979,0.000000,0.151979,0.338643,0.000000
8,0.000000,0.000000,0.211485,0.000000,0.000000,0.000000,0.211485,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.422970,0.000000,0.211485,0.000000,0.000000,0.000000,0.211485,0.000000
9,0.000000,0.000000,0.000000,0.629402,0.376624,0.000000,0.000000,0.000000,0.209801,0.000000,...,0.000000,0.000000,0.000000,0.209801,0.000000,0.000000,0.000000,0.188312,0.000000,0.209801


Code di atas bertujuan untuk mengonversi hasil vektorisasi TF-IDF ke dalam sebuah DataFrame. Ini memungkinkan kita untuk lebih mudah memahami dan menganalisis hasil vektorisasi.

**Langkah-langkah yang dilakukan:**

1. **Inisialisasi DataFrame**: Kita membuat sebuah objek DataFrame yang disebut `df2` dengan menggunakan library pandas. DataFrame ini akan berisi hasil vektorisasi TF-IDF.

2. **Mengonversi Vektor TF-IDF**: Kita menggunakan metode `.toarray()` untuk mengonversi matriks hasil vektorisasi TF-IDF `X_tfidf` menjadi sebuah array Numpy. Kemudian, array ini digunakan sebagai data untuk mengisi DataFrame `df2`.

3. **Mengatur Nama Kolom**: Nama-nama kolom DataFrame diambil dari fitur-fitur (kata-kata) yang digunakan dalam vektorisasi TF-IDF. Nama-nama kolom ini diambil dari objek `tfidf_vectorizer` menggunakan metode `.get_feature_names_out()`.

4. **Mengatur Index**: Index DataFrame diambil dari indeks data awalnya (dalam hal ini, nomor baris data dari `data`).

Hasil dari langkah ini adalah DataFrame `df2` yang berisi representasi TF-IDF dari teks pada setiap baris data. Setiap kolom dalam DataFrame ini mewakili kata-kata dalam bentuk TF-IDF, dan setiap baris mewakili satu dokumen atau sampel teks dalam dataset. Dengan DataFrame ini, Anda dapat dengan mudah menjelajahi, menganalisis, atau memproses data teks lebih lanjut.

## 5. Dataset Spliting

Pemisahan Dataset (Dataset Splitting) adalah salah satu tahap penting dalam proses pembuatan model machine learning. Tujuannya adalah membagi dataset menjadi dua bagian utama: data pelatihan (training data) dan data pengujian (testing data). Dataset pelatihan digunakan untuk melatih model, sedangkan dataset pengujian digunakan untuk menguji sejauh mana model tersebut mampu melakukan prediksi yang akurat.

Teknik split validation adalah salah satu metode pemisahan dataset yang umum digunakan. Dalam metode ini, dataset dibagi menjadi dua bagian, biasanya dalam rasio tertentu, seperti 80% data pelatihan dan 20% data pengujian. Namun, rasio ini dapat bervariasi tergantung pada tugas dan jumlah data yang Anda miliki. Data pelatihan digunakan untuk melatih model machine learning, sementara data pengujian digunakan untuk menguji kinerja model.

Biasanya, data diacak sebelum dibagi untuk memastikan representasi yang merata di antara data pelatihan dan data pengujian. Metode split validation membantu mencegah overfitting (ketika model terlalu beradaptasi dengan data pelatihan) dan memungkinkan kita untuk mengukur kinerja model pada data yang belum pernah dilihat sebelumnya.

Saat mengimplementasikan split validation dalam kode, kita biasanya menggunakan fungsi atau metode dari library seperti scikit-learn, yang memudahkan proses pembagian dataset dengan rasio yang sesuai.

In [ ]:
# Menggunakan kolom 'Sentiment' sebagai target
y = data['Label']

# Pemisahan fitur (X)
X_count_train, X_count_test, y_train, y_test = train_test_split(X_count, y, test_size=0.2, random_state=42) # Pemisahan Feature untuk BOW
X_tfidf_train, X_tfidf_test, _, _ = train_test_split(X_tfidf, y, test_size=0.2, random_state=42) # Pemisahan feature untuk TF-IDF

Code ini adalah bagian dari proses pemisahan dataset menjadi data pelatihan (training data) dan data pengujian (testing data) untuk digunakan dalam model machine learning. Proses pemisahan ini penting untuk menguji kinerja model pada data yang belum pernah dilihat sebelumnya.

Langkah-langkahnya adalah sebagai berikut:

1. **Menggunakan kolom 'Sentiment' sebagai target**: Di sini, Anda mengambil kolom 'Label' dari dataset dan menjadikannya sebagai target yang akan dihasilkan oleh model machine learning. Target ini akan digunakan untuk melatih model agar dapat memprediksi sentimen dari data teks.

2. **Pemisahan Fitur (X)**: Data fitur, yang berisi representasi teks (BOW dan TF-IDF) dari berita, perlu dipisahkan menjadi dua set, satu untuk BOW (X_count) dan satu untuk TF-IDF (X_tfidf).

3. **`train_test_split`**: Fungsi `train_test_split` dari library scikit-learn digunakan untuk membagi dataset menjadi data pelatihan dan pengujian. Parameter yang digunakan dalam fungsi ini adalah:
   - `X_count` dan `X_tfidf`: Data fitur yang telah diperoleh dari representasi BOW dan TF-IDF.
   - `y`: Target yang telah ditentukan sebelumnya (kolom 'Label').
   - `test_size=0.2`: Menunjukkan bahwa Anda ingin membagi dataset menjadi data pengujian sebanyak 20% dari total data.
   - `random_state=42`: Digunakan untuk mengatur pembagian secara acak, namun dengan seed (42) yang sama, sehingga hasilnya dapat direproduksi jika perlu.

Hasil dari pemisahan ini adalah empat set data:
- `X_count_train`: Data pelatihan BOW.
- `X_count_test`: Data pengujian BOW.
- `X_tfidf_train`: Data pelatihan TF-IDF.
- `X_tfidf_test`: Data pengujian TF-IDF.

Dengan melakukan pemisahan ini, Anda siap untuk melatih model machine learning (misalnya, SVM atau Random Forest) dan mengukur kinerjanya pada data pengujian yang tidak pernah dilihat sebelumnya. Ini adalah langkah penting dalam pengembangan model sentimen analysis.

## 6. Klasifikasi Model Metode SVM

Metode Support Vector Machine (SVM) adalah salah satu algoritma klasifikasi dalam pembelajaran mesin. Tujuan utama SVM adalah memisahkan dua kelas dengan cara menemukan "batas keputusan" yang optimal. Batas keputusan ini disebut "hyperplane" dan dipilih sedemikian rupa sehingga memiliki jarak maksimum dari titik-titik pelatihan yang termasuk dalam dua kelas yang berbeda. Hyperplane ini menjadi garis atau permukaan yang memisahkan data ke dalam dua kelas.

Beberapa poin penting tentang SVM:

1. **Klasifikasi Biner**: SVM awalnya dikembangkan untuk klasifikasi biner, yang berarti memisahkan dua kelas. Namun, dengan beberapa modifikasi, SVM juga dapat digunakan untuk masalah klasifikasi multikelas.

2. **Margin Maksimum**: SVM berusaha untuk mencari hyperplane yang memiliki margin maksimum, yaitu jarak terdekat antara hyperplane dan titik pelatihan terjauh dari kedua kelas. Ini disebut sebagai "maksimisasi margin."

3. **Fungsi Kernel**: SVM memungkinkan penggunaan fungsi kernel, seperti kernel linear atau kernel non-linear (seperti kernel polinomial atau RBF), untuk menangani data yang tidak dapat dipisahkan secara linear dalam ruang fitur asli.

4. **Regresi**: Selain klasifikasi, SVM juga dapat digunakan untuk regresi dengan tujuan mencari hyperplane yang memiliki "margin epsilon" yang berisi sebagian besar data pelatihan.

Dalam tahap ini, Anda menggunakan metode SVM untuk melatih model klasifikasi menggunakan dua representasi teks yang berbeda (BOW dan TF-IDF) dan kemudian mengukur kinerjanya. Selanjutnya, Anda dapat menjelaskan hasil dari penggunaan SVM pada dataset yang Anda miliki.

### Klasifikasi Model SVM dengan Fitur Ektraction BOW

In [ ]:
# Pelatihan Model
svm_model = SVC()
svm_model.fit(X_count_train, y_train)
y_pred_svm = svm_model.predict(X_count_test)
y_pred_svm

array(['Olahraga', 'Kesehatan', 'Politik'], dtype=object)

Code di atas adalah implementasi dari pelatihan model menggunakan Support Vector Machine (SVM) untuk melakukan klasifikasi teks. Berikut adalah penjelasan lengkapnya:


1. `svm_model = SVC()`: Saya membuat sebuah objek Support Vector Machine (SVM) dengan menginisialisasinya menggunakan kelas `SVC` (Support Vector Classification) dari pustaka scikit-learn (sklearn). Langkah ini adalah awal dari proses pembuatan model klasifikasi SVM.

2. `svm_model.fit(X_count_train, y_train)`: Saya menggunakan objek SVM untuk melatih model. `X_count_train` adalah himpunan data pelatihan yang berisi representasi BOW dari teks yang akan saya gunakan untuk melatih model. `y_train` adalah himpunan label yang sesuai dengan data pelatihan. Model ini akan mempelajari pola dalam data pelatihan untuk membedakan teks-teks berdasarkan labelnya.

3. `y_pred_svm = svm_model.predict(X_count_test)`: Setelah model SVM dilatih, saya menggunakan model tersebut untuk membuat prediksi terhadap himpunan data uji. `X_count_test` adalah himpunan data uji yang berisi representasi BOW dari teks yang akan digunakan untuk melakukan prediksi. Hasil prediksi akan disimpan dalam variabel `y_pred_svm`.

4. `y_pred_svm`: Saya mencetak hasil prediksi yang dihasilkan oleh model SVM ke layar. Hasilnya adalah sebuah array yang berisi label-label yang mewakili kategori-kategori yang diprediksi untuk setiap teks dalam himpunan data uji.

Hasil yang saya tunjukkan adalah contoh hasil prediksi. Array tersebut berisi kategori-kategori yang diprediksi untuk teks-teks dalam himpunan data uji dengan ekstraksi fitur BOW. Model SVM mengklasifikasikan teks-teks tersebut ke dalam salah satu dari kategori tersebut berdasarkan pembelajaran dari data pelatihan.kasikan teks-teks tersebut ke dalam salah satu dari tiga kategori tersebut.

In [ ]:
# Evaluasi Model SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)

confusion_svm = confusion_matrix(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm, target_names=data['Label'].unique())

print("Confusion Matrix - SVM:")
print(confusion_svm)

print("\nClassification Report - SVM:")
print(report_svm)

print("\nNilai Akurasi - SVM:")
print(accuracy_svm)

Confusion Matrix - SVM:
[[1 0 0]
 [0 0 0]
 [0 1 1]]

Classification Report - SVM:
              precision    recall  f1-score   support

     Politik       1.00      1.00      1.00         1
    Olahraga       0.00      0.00      0.00         0
   Kesehatan       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.67      0.50      0.56         3
weighted avg       1.00      0.67      0.78         3


Nilai Akurasi - SVM:
0.6666666666666666


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Penjelasan mengenai code evaluasi untuk algoritma SVM metode Feature Ekstraction BOW



1. `accuracy_svm = accuracy_score(y_test, y_pred_svm)`: Saya menggunakan metode `accuracy_score` dari scikit-learn untuk menghitung akurasi model SVM. Akurasi adalah metrik yang mengukur sejauh mana model berhasil dalam melakukan prediksi yang benar. Saya membandingkan label sebenarnya dalam himpunan data uji (`y_test`) dengan hasil prediksi yang dihasilkan oleh model SVM (`y_pred_svm`). Hasil akurasi disimpan dalam variabel `accuracy_svm`.

2. `confusion_svm = confusion_matrix(y_test, y_pred_svm)`: Saya menggunakan metode `confusion_matrix` dari scikit-learn untuk menghitung matriks kebingungan (confusion matrix) untuk model SVM. Matriks kebingungan adalah tabel yang memungkinkan saya untuk melihat seberapa baik model mengklasifikasikan data dalam berbagai kategori. Ini mencakup informasi tentang true positive, false positive, true negative, dan false negative. Hasil matriks kebingungan disimpan dalam variabel `confusion_svm`.

3. `report_svm = classification_report(y_test, y_pred_svm, target_names=data['Label'].unique())`: Saya menggunakan metode `classification_report` dari scikit-learn untuk membuat laporan klasifikasi yang mencakup sejumlah metrik evaluasi, seperti presisi, recall, f1-score, dan lainnya. Laporan klasifikasi ini disusun berdasarkan perbandingan antara label sebenarnya dalam himpunan data uji (`y_test`) dan hasil prediksi model SVM (`y_pred_svm`). Saya juga menggunakan `target_names` untuk memberikan nama kategori yang sesuai dengan data Anda.

4. `print("Confusion Matrix - SVM:")` dan `print(confusion_svm)`: Saya mencetak matriks kebingungan ke layar agar saya bisa melihat bagaimana model SVM melakukan klasifikasi pada setiap kategori. Matriks kebingungan akan menampilkan berapa banyak prediksi yang benar dan berapa banyak prediksi yang salah untuk setiap kategori.

5. `print("\nClassification Report - SVM:")` dan `print(report_svm)`: Saya mencetak laporan klasifikasi ke layar. Laporan klasifikasi memberikan wawasan lebih mendalam tentang kinerja model, termasuk presisi, recall, f1-score, dan lainnya, untuk setiap kategori.

6. `print("\nNilai Akurasi - SVM:")` dan `print(accuracy_svm)`: Saya mencetak akurasi model ke layar. Akurasi adalah metrik yang mencerminkan sejauh mana model berhasil dalam melakukan klasifikasi dengan benar.

Hasil dari evaluasi ini akan membantu saya memahami sejauh mana model SVM yang menggunakan ekstraksi fitur BOW berhasil dalam melakukan klasifikasi pada data uji dan apakah perlu dilakukan peningkatan atau penyesuaian pada model tersebut.

Hasil dari evaluasi model yang Anda lakukan dengan ekstraksi fitur Bag of Words (BOW) adalah sebagai berikut:



1. **Confusion Matrix - SVM**:
   - Baris pertama mewakili kategori "Politik." Model SVM saya berhasil mengklasifikasikan 1 sampel ke dalam kategori "Politik" dengan benar.
   - Baris ketiga mewakili kategori "Kesehatan." Model SVM saya berhasil mengklasifikasikan 1 dari 2 sampel ke dalam kategori "Kesehatan."

   Namun, terdapat satu sampel yang seharusnya termasuk dalam kategori "Olahraga," tetapi model saya tidak dapat mengklasifikasikannya dengan benar, sehingga matriks kebingungannya memiliki nilai 0 untuk kategori "Olahraga."

2. **Classification Report - SVM**:
   - **Presisi (precision)** adalah metrik yang mengukur sejauh mana prediksi positif model saya benar. Untuk kategori "Politik" dan "Kesehatan," presisi adalah 1.00, yang berarti semua prediksi positif model saya benar. Namun, untuk kategori "Olahraga," presisi adalah 0.00 karena model saya tidak membuat satu pun prediksi positif untuk kategori tersebut.
   - **Recall (recall)** adalah metrik yang mengukur sejauh mana model saya berhasil menangkap semua contoh positif yang sebenarnya. Kategori "Politik" memiliki recall 1.00, yang berarti model saya berhasil menangkap semua sampel positif. Kategori "Olahraga" memiliki recall 0.00 karena tidak ada sampel positif yang berhasil diidentifikasi. Kategori "Kesehatan" memiliki recall 0.50, yang berarti model saya berhasil menangkap setengah dari sampel positif.
   - **F1-score (f1-score)** adalah metrik yang menggabungkan presisi dan recall menjadi satu angka. Untuk kategori "Politik," F1-score adalah 1.00, yang mengindikasikan kinerja yang baik. Namun, kategori "Olahraga" memiliki F1-score 0.00 karena presisi dan recall keduanya adalah 0. Kategori "Kesehatan" memiliki F1-score sebesar 0.67.
   - **Support** adalah jumlah sampel yang sesuai dengan masing-masing kategori dalam himpunan data uji.

3. **Nilai Akurasi - SVM**: Akurasi model adalah 0.67, yang berarti model ini mampu memprediksi dengan benar sekitar 67% dari semua sampel dalam himpunan data uji.

Perlu diperhatikan bahwa model memiliki kesulitan dalam mengklasifikasikan kategori "Olahraga," yang tidak memiliki satu pun sampel positif dalam himpunan data uji. Hal ini mengakibatkan beberapa metrik, seperti presisi, recall, dan F1-score, menjadi tidak terdefinisi (0.00) untuk kategori tersebut. Selain itu, akurasi yang tinggi sebagian besar disebabkan oleh kinerja yang sangat baik dalam mengklasifikasikan kategori "Politik," sedangkan kategori "Kesehatan" memiliki performa yang lebih menengah. Evaluasi model ini memberikan informasi tentang kinerja model yang perlu ditingkatkan, terutama dalam menghadapi kategori dengan sampel yang lebih sedikit atau tidak ada.

###Klasifikasi Model SVM dengan Fitur Ektraction TF-IDF

In [ ]:
# Pelatihan Model
svm_model_2 = SVC()
svm_model_2.fit(X_tfidf_train, y_train)
y_pred_svm_2 = svm_model_2.predict(X_tfidf_test)
y_pred_svm_2

array(['Politik', 'Kesehatan', 'Politik'], dtype=object)

Kode di atas adalah untuk melatih model SVM dengan ekstraksi fitur Term Frequency-Inverse Document Frequency (TF-IDF) dan kemudian melakukan prediksi menggunakan model tersebut. Berikut adalah penjelasan lengkapnya:


1. `svm_model_2 = SVC()`: Saya membuat sebuah objek Support Vector Machine (SVM) dengan menginisialisasinya menggunakan kelas `SVC` (Support Vector Classification) dari pustaka scikit-learn. Ini merupakan langkah awal dalam pembuatan model SVM untuk klasifikasi dengan ekstraksi fitur TF-IDF.

2. `svm_model_2.fit(X_tfidf_train, y_train)`: Saya menggunakan objek SVM (`svm_model_2`) untuk melatih model. `X_tfidf_train` adalah himpunan data pelatihan yang berisi representasi TF-IDF dari teks yang akan saya gunakan untuk melatih model. `y_train` adalah himpunan label yang sesuai dengan data pelatihan. Model ini akan mempelajari pola dalam data pelatihan untuk membedakan teks-teks berdasarkan labelnya dengan menggunakan fitur TF-IDF.

3. `y_pred_svm_2 = svm_model_2.predict(X_tfidf_test)`: Setelah model SVM dilatih dengan fitur TF-IDF, saya menggunakan model tersebut untuk membuat prediksi terhadap himpunan data uji. `X_tfidf_test` adalah himpunan data uji yang berisi representasi TF-IDF dari teks yang akan digunakan untuk melakukan prediksi. Hasil prediksi akan disimpan dalam variabel `y_pred_svm_2`.

4. `y_pred_svm_2`: Ini adalah hasil prediksi yang dihasilkan oleh model SVM yang menggunakan fitur TF-IDF. Hasilnya adalah sebuah array yang berisi label-label yang mewakili kategori-kategori yang diprediksi untuk setiap teks dalam himpunan data uji. Model SVM dengan ekstraksi fitur TF-IDF mengklasifikasikan teks-teks tersebut ke dalam salah satu dari kategori tersebut.

Dengan kode ini, saya telah melatih model SVM menggunakan representasi TF-IDF dari teks dan melakukan prediksi terhadap data uji dengan model yang telah dilatih. Hal ini memungkinkan saya untuk mengevaluasi kinerja model SVM dengan ekstraksi fitur TF-IDF terhadap data uji saya.

In [ ]:
# Evaluasi Model SVM
accuracy_svm_2 = accuracy_score(y_test, y_pred_svm_2)

confusion_svm_2 = confusion_matrix(y_test, y_pred_svm_2)
report_svm_2 = classification_report(y_test, y_pred_svm_2, target_names=['Kesehatan', 'Politik'])

print("Confusion Matrix - SVM:")
print(confusion_svm_2)

print("\nClassification Report - SVM:")
print(report_svm_2)

print("\nNilai Akurasi - SVM:")
print(accuracy_svm_2)

Confusion Matrix - SVM:
[[1 0]
 [0 2]]

Classification Report - SVM:
              precision    recall  f1-score   support

   Kesehatan       1.00      1.00      1.00         1
     Politik       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Nilai Akurasi - SVM:
1.0


Berikut penjelasan lengkap mengenai kode evaluasi model SVM dengan ekstraksi fitur Term Frequency-Inverse Document Frequency (TF-IDF):

1. `accuracy_svm_2 = accuracy_score(y_test, y_pred_svm_2)`: Saya menggunakan metode `accuracy_score` dari scikit-learn untuk menghitung akurasi model SVM yang menggunakan ekstraksi fitur TF-IDF. Akurasi adalah metrik yang mengukur sejauh mana model berhasil dalam melakukan prediksi yang benar. Saya membandingkan label sebenarnya dalam himpunan data uji (`y_test`) dengan hasil prediksi yang dihasilkan oleh model SVM (`y_pred_svm_2`). Hasil akurasi disimpan dalam variabel `accuracy_svm_2`.

2. `confusion_svm_2 = confusion_matrix(y_test, y_pred_svm_2)`: Saya menggunakan metode `confusion_matrix` dari scikit-learn untuk menghitung matriks kebingungan (confusion matrix) untuk model SVM dengan ekstraksi fitur TF-IDF. Matriks kebingungan adalah tabel yang memungkinkan saya untuk melihat seberapa baik model mengklasifikasikan data dalam berbagai kategori. Hasil matriks kebingungan disimpan dalam variabel `confusion_svm_2`.

3. `report_svm_2 = classification_report(y_test, y_pred_svm_2, target_names=['Kesehatan', 'Politik'])`: Saya menggunakan metode `classification_report` dari scikit-learn untuk membuat laporan klasifikasi yang mencakup sejumlah metrik evaluasi, seperti presisi, recall, f1-score, dan lainnya. Laporan klasifikasi ini disusun berdasarkan perbandingan antara label sebenarnya dalam himpunan data uji (`y_test`) dengan hasil prediksi model SVM (`y_pred_svm_2`). Saya juga menggunakan `target_names` untuk memberikan nama kategori yang sesuai dengan data saya, yaitu 'Kesehatan' dan 'Politik'.

4. `print("Confusion Matrix - SVM:")` dan `print(confusion_svm_2)`: Saya mencetak matriks kebingungan ke layar agar saya bisa melihat bagaimana model SVM dengan ekstraksi fitur TF-IDF melakukan klasifikasi pada setiap kategori. Matriks kebingungan akan menampilkan berapa banyak prediksi yang benar dan berapa banyak prediksi yang salah untuk masing-masing kategori.

5. `print("\nClassification Report - SVM:")` dan `print(report_svm_2)`: Saya mencetak laporan klasifikasi ke layar. Laporan klasifikasi memberikan wawasan lebih mendalam tentang kinerja model, termasuk presisi, recall, f1-score, dan lainnya, untuk masing-masing kategori yang telah ditentukan.

6. `print("\nNilai Akurasi - SVM:")` dan `print(accuracy_svm_2)`: Saya mencetak akurasi model ke layar. Akurasi adalah metrik yang mencerminkan sejauh mana model berhasil dalam melakukan klasifikasi dengan benar.

Dengan evaluasi ini, saya dapat memahami sejauh mana model SVM yang menggunakan ekstraksi fitur TF-IDF berhasil dalam melakukan klasifikasi pada data uji dan apakah perlu dilakukan peningkatan atau penyesuaian pada model tersebut. Evaluasi ini memberikan gambaran tentang kinerja model dalam membedakan antara kategori 'Kesehatan' dan 'Politik' dengan ekstraksi fitur TF-IDF.

Hasil evaluasi model SVM dengan ekstraksi fitur Term Frequency-Inverse Document Frequency (TF-IDF) yang Anda lakukan adalah sebagai berikut:

**Confusion Matrix - SVM**:
- Terdapat dua kategori yang diprediksi, yaitu "Kesehatan" dan "Politik."
- Untuk kategori "Kesehatan," model berhasil mengklasifikasikan 1 sampel dengan benar, dan tidak ada prediksi yang salah (0).
- Untuk kategori "Politik," model berhasil mengklasifikasikan 2 sampel dengan benar, dan tidak ada prediksi yang salah (0).

**Classification Report - SVM**:
- **Presisi (precision)**, **recall**, dan **F1-score** untuk kedua kategori ("Kesehatan" dan "Politik") adalah 1.00. Ini mengindikasikan bahwa model berhasil dalam memprediksi dengan sempurna untuk kedua kategori.
- Jumlah sampel dalam kategori "Kesehatan" adalah 1, sedangkan dalam kategori "Politik" adalah 2.
- **Akurasi (accuracy)** adalah 1.0 (100%), yang berarti model ini berhasil memprediksi dengan benar semua sampel dalam himpunan data uji.


Selain itu, bahwa dalam hasil prediksi sebelumnya tidak ada label 'Olahraga' yang muncul. Oleh karena itu, dalam hasil evaluasi model ini, hanya dua kategori yang dievaluasi, yaitu 'Kesehatan' dan 'Politik.' Model ini tampaknya memprediksi dengan sempurna untuk kedua kategori tersebut dalam data uji yang digunakan.

## 7. Klasifikasi Model Metode Random Forest

Tentu, mari kita mulai dengan pengenalan mengenai apa itu Random Forest.

**Random Forest** adalah salah satu metode dalam *machine learning* yang digunakan untuk melakukan klasifikasi, regresi, dan pemrosesan data. Random Forest adalah jenis algoritma ensemble, yang berarti ia menggabungkan hasil dari beberapa model (pohon keputusan) untuk meningkatkan kinerja prediksi.

Konsep utama dalam Random Forest adalah penggunaan banyak pohon keputusan yang dibangun secara acak. Setiap pohon keputusan ini merupakan model yang mengambil keputusan berdasarkan aturan yang dipelajari dari data pelatihan. Dalam Random Forest, pohon-pohon ini dibangun secara independen dan kemudian digabungkan untuk menghasilkan hasil akhir.

Berikut adalah beberapa karakteristik utama dari Random Forest:

1. **Bootstrap Aggregating (Bagging)**: Random Forest menggunakan teknik *bootstrap aggregating* di mana setiap pohon dibangun berdasarkan sampel acak dari data pelatihan dengan penggantian. Hal ini memungkinkan setiap pohon memiliki keragaman yang lebih besar.

2. **Pemilihan Fitur Acak**: Ketika membangun setiap pohon, hanya sebagian kecil dari fitur yang digunakan. Ini memastikan keragaman lebih lanjut di antara pohon-pohon.

3. **Voting atau Average**: Hasil dari setiap pohon digabungkan dengan cara melakukan voting (klasifikasi) atau rata-rata (regresi) untuk menghasilkan hasil akhir.

4. **Tangguh Terhadap Overfitting**: Random Forest cenderung tangguh terhadap overfitting karena kombinasi dari beberapa pohon yang mengurangi kemungkinan kesalahan dari satu pohon individu.

Random Forest sangat populer dalam berbagai aplikasi seperti klasifikasi teks, pengenalan gambar, analisis bioinformatika, dan banyak lagi.

### Klasifikasi Model Random Forest dengan Fitur Ektraction BOW

In [ ]:
# Membangun Model Random Forest Dengan BOW
rf_model = RandomForestClassifier()
rf_model.fit(X_count_train, y_train)
y_pred_rf = rf_model.predict(X_count_test)
y_pred_rf

array(['Politik', 'Kesehatan', 'Politik'], dtype=object)



Kode di atas adalah untuk membangun model klasifikasi Random Forest dengan menggunakan ekstraksi fitur Bag of Words (BOW) dan melakukan prediksi dengan model tersebut. Berikut penjelasan lengkapnya:

1. `rf_model = RandomForestClassifier()`: Saya membuat objek model Random Forest dengan menginisialisasinya menggunakan kelas `RandomForestClassifier` dari pustaka scikit-learn (sklearn). Model ini akan saya gunakan untuk melakukan klasifikasi.

2. `rf_model.fit(X_count_train, y_train)`: Saya menggunakan objek model Random Forest (`rf_model`) untuk melatih model. `X_count_train` adalah himpunan data pelatihan yang berisi representasi Bag of Words (BOW) dari teks yang akan saya gunakan untuk melatih model. `y_train` adalah himpunan label yang sesuai dengan data pelatihan. Model Random Forest akan mempelajari pola dalam data pelatihan untuk membedakan teks-teks berdasarkan labelnya.

3. `y_pred_rf = rf_model.predict(X_count_test)`: Setelah model Random Forest dilatih dengan fitur BOW, saya menggunakan model tersebut untuk membuat prediksi terhadap himpunan data uji. `X_count_test` adalah himpunan data uji yang berisi representasi BOW dari teks yang akan saya gunakan untuk melakukan prediksi. Hasil prediksi akan disimpan dalam variabel `y_pred_rf`.

4. `y_pred_rf`: Ini adalah hasil prediksi yang dihasilkan oleh model Random Forest yang menggunakan fitur BOW. Hasilnya adalah sebuah array yang berisi label-label yang mewakili kategori-kategori yang diprediksi untuk setiap teks dalam himpunan data uji. Model Random Forest mengklasifikasikan teks-teks tersebut ke dalam salah satu dari kategori tersebut.

Hasil prediksi dari kode tersebut adalah sebagai berikut:
```
array(['Politik', 'Kesehatan', 'Politik'], dtype=object)
```

Hasil ini menunjukkan bahwa model Random Forest telah melakukan prediksi kategori untuk tiga teks dalam himpunan data uji. Teks pertama diprediksi sebagai "Politik," teks kedua diprediksi sebagai "Kesehatan," dan teks ketiga juga diprediksi sebagai "Politik."

Dengan demikian, saya telah berhasil membangun model Random Forest dengan ekstraksi fitur BOW dan melakukan prediksi dengan model tersebut. Model ini digunakan untuk mengklasifikasikan teks-teks dalam himpunan data uji ke dalam kategori yang sesuai berdasarkan representasi Bag of Words.

In [ ]:
# Evaluasi Model Random Forest Dengan BOW
accuracy_rf= accuracy_score(y_test, y_pred_rf)

confusion_rf= confusion_matrix(y_test, y_pred_rf)
report_rf= classification_report(y_test, y_pred_rf, target_names=['Kesehatan', 'Politik'])

print("Confusion Matrix - Random Forest:")
print(confusion_rf)

print("\nClassification Report - Random Forest:")
print(report_rf)

print("\nNilai Akurasi - Random Forest:")
print(accuracy_rf)

Confusion Matrix - Random Forest:
[[1 0]
 [0 2]]

Classification Report - Random Forest:
              precision    recall  f1-score   support

   Kesehatan       1.00      1.00      1.00         1
     Politik       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Nilai Akurasi - Random Forest:
1.0


Berikut penjelasan lengkap mengenai kode evaluasi model Random Forest dengan ekstraksi fitur Bag of Words (BOW):

1. `accuracy_rf = accuracy_score(y_test, y_pred_rf)`: Saya menggunakan metode `accuracy_score` dari scikit-learn untuk menghitung akurasi model Random Forest yang menggunakan ekstraksi fitur BOW. Akurasi adalah metrik yang mengukur sejauh mana model berhasil dalam melakukan prediksi yang benar. Saya membandingkan label sebenarnya dalam himpunan data uji (`y_test`) dengan hasil prediksi yang dihasilkan oleh model Random Forest (`y_pred_rf`). Hasil akurasi disimpan dalam variabel `accuracy_rf`.

2. `confusion_rf = confusion_matrix(y_test, y_pred_rf)`: Saya menggunakan metode `confusion_matrix` dari scikit-learn untuk menghitung matriks kebingungan (confusion matrix) untuk model Random Forest dengan ekstraksi fitur BOW. Matriks kebingungan adalah tabel yang memungkinkan saya untuk melihat seberapa baik model mengklasifikasikan data dalam berbagai kategori. Hasil matriks kebingungan disimpan dalam variabel `confusion_rf`.

3. `report_rf = classification_report(y_test, y_pred_rf, target_names=['Kesehatan', 'Politik'])`: Saya menggunakan metode `classification_report` dari scikit-learn untuk membuat laporan klasifikasi yang mencakup sejumlah metrik evaluasi, seperti presisi, recall, f1-score, dan lainnya. Laporan klasifikasi ini disusun berdasarkan perbandingan antara label sebenarnya dalam himpunan data uji (`y_test`) dengan hasil prediksi model Random Forest (`y_pred_rf`). Saya juga menggunakan `target_names` untuk memberikan nama kategori yang sesuai dengan data Anda, yaitu 'Kesehatan' dan 'Politik'.

4. `print("Confusion Matrix - Random Forest:")` dan `print(confusion_rf)`: Saya mencetak matriks kebingungan ke layar agar saya bisa melihat bagaimana model Random Forest dengan ekstraksi fitur BOW melakukan klasifikasi pada setiap kategori. Matriks kebingungan akan menampilkan berapa banyak prediksi yang benar dan berapa banyak prediksi yang salah untuk masing-masing kategori.

5. `print("\nClassification Report - Random Forest:")` dan `print(report_rf)`: Saya mencetak laporan klasifikasi ke layar. Laporan klasifikasi memberikan wawasan lebih mendalam tentang kinerja model, termasuk presisi, recall, f1-score, dan lainnya, untuk masing-masing kategori yang telah ditentukan.

6. `print("\nNilai Akurasi - Random Forest:")` dan `print(accuracy_rf)`: Saya mencetak akurasi model ke layar. Akurasi adalah metrik yang mencerminkan sejauh mana model berhasil dalam melakukan klasifikasi dengan benar.

Dengan evaluasi ini, saya dapat memahami sejauh mana model Random Forest yang menggunakan ekstraksi fitur BOW berhasil dalam melakukan klasifikasi pada data uji dan apakah perlu dilakukan peningkatan atau penyesuaian pada model tersebut. Evaluasi ini memberikan gambaran tentang kinerja model dalam membedakan antara kategori 'Kesehatan' dan 'Politik' dengan ekstraksi fitur Bag of Words.

Hasil evaluasi model Random Forest dengan ekstraksi fitur Bag of Words adalah sebagai berikut:

**Confusion Matrix - Random Forest**:
- Terdapat dua kategori yang dievaluasi, yaitu "Kesehatan" dan "Politik."
- Untuk kategori "Kesehatan," model berhasil mengklasifikasikan 1 sampel dengan benar, dan tidak ada prediksi yang salah (0).
- Untuk kategori "Politik," model berhasil mengklasifikasikan 2 sampel dengan benar, dan tidak ada prediksi yang salah (0).

**Classification Report - Random Forest**:
- **Presisi (precision)**, **recall**, dan **F1-score** untuk kedua kategori ("Kesehatan" dan "Politik") adalah 1.00. Ini mengindikasikan bahwa model berhasil dalam memprediksi dengan sempurna untuk kedua kategori.
- Jumlah sampel dalam kategori "Kesehatan" adalah 1, sedangkan dalam kategori "Politik" adalah 2.
- **Akurasi (accuracy)** adalah 1.0 (100%), yang berarti model ini berhasil memprediksi dengan benar semua sampel dalam himpunan data uji.

Hasil ini menunjukkan bahwa model Random Forest dengan ekstraksi fitur Bag of Words (BOW) telah melakukan prediksi dengan sempurna untuk kedua kategori yang dievaluasi. Semua sampel dalam himpunan data uji telah diklasifikasikan dengan benar.

###Klasifikasi Model Random Forest dengan Fitur Ektraction TF-IDF

In [ ]:
# Membangun Model Random Forest Dengan TF-IDF
rf_model_2 = RandomForestClassifier()
rf_model_2.fit(X_tfidf_train, y_train)
y_pred_rf_2 = rf_model_2.predict(X_tfidf_test)
y_pred_rf_2

array(['Politik', 'Kesehatan', 'Politik'], dtype=object)

Kode di atas adalah untuk membangun model klasifikasi Random Forest dengan menggunakan ekstraksi fitur Term Frequency-Inverse Document Frequency (TF-IDF) dan melakukan prediksi dengan model tersebut. Berikut penjelasan lengkapnya:

1. `rf_model_2 = RandomForestClassifier()`: Saya membuat objek model Random Forest dengan menginisialisasinya menggunakan kelas `RandomForestClassifier` dari pustaka scikit-learn (sklearn). Model ini akan digunakan untuk melakukan klasifikasi.

2. `rf_model_2.fit(X_tfidf_train, y_train)`: Saya menggunakan objek model Random Forest (`rf_model_2`) untuk melatih model. `X_tfidf_train` adalah himpunan data pelatihan yang berisi representasi TF-IDF dari teks yang akan digunakan untuk melatih model. `y_train` adalah himpunan label yang sesuai dengan data pelatihan. Model Random Forest akan mempelajari pola dalam data pelatihan untuk membedakan teks-teks berdasarkan labelnya.

3. `y_pred_rf_2 = rf_model_2.predict(X_tfidf_test)`: Setelah model Random Forest dilatih dengan fitur TF-IDF, saya menggunakan model tersebut untuk membuat prediksi terhadap himpunan data uji. `X_tfidf_test` adalah himpunan data uji yang berisi representasi TF-IDF dari teks yang akan saya gunakan untuk melakukan prediksi. Hasil prediksi akan disimpan dalam variabel `y_pred_rf_2`.

4. `y_pred_rf_2`: Ini adalah hasil prediksi yang dihasilkan oleh model Random Forest yang menggunakan fitur TF-IDF. Hasilnya adalah sebuah array yang berisi label-label yang mewakili kategori-kategori yang diprediksi untuk setiap teks dalam himpunan data uji. Model Random Forest mengklasifikasikan teks-teks tersebut ke dalam salah satu dari kategori tersebut.

Hasil prediksi dari kode tersebut adalah sebagai berikut:
```
array(['Politik', 'Kesehatan', 'Politik'], dtype=object)
```

Hasil ini menunjukkan bahwa model Random Forest telah melakukan prediksi kategori untuk tiga teks dalam himpunan data uji. Teks pertama diprediksi sebagai "Politik," teks kedua diprediksi sebagai "Kesehatan," dan teks ketiga juga diprediksi sebagai "Politik."

Dengan demikian, saya telah berhasil membangun model Random Forest dengan ekstraksi fitur TF-IDF dan melakukan prediksi dengan model tersebut. Model ini digunakan untuk mengklasifikasikan teks-teks dalam himpunan data uji ke dalam kategori yang sesuai berdasarkan representasi TF-IDF.

In [ ]:
# Evaluasi Model Random Forest Dengan TF=IDF
accuracy_rf_2= accuracy_score(y_test, y_pred_rf_2 )

confusion_rf_2= confusion_matrix(y_test, y_pred_rf_2 )
report_rf_2= classification_report(y_test, y_pred_rf_2 , target_names=['Kesehatan', 'Politik'])

print("Confusion Matrix - Random Forest:")
print(confusion_rf_2)

print("\nClassification Report - Random Forest:")
print(report_rf_2)

print("\nNilai Akurasi - Random Forest:")
print(accuracy_rf_2)

Confusion Matrix - Random Forest:
[[1 0]
 [0 2]]

Classification Report - Random Forest:
              precision    recall  f1-score   support

   Kesehatan       1.00      1.00      1.00         1
     Politik       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Nilai Akurasi - Random Forest:
1.0


Berikut adalah penjelasan lengkap mengenai kode evaluasi model Random Forest dengan ekstraksi fitur TF-IDF:

1. `accuracy_rf_2 = accuracy_score(y_test, y_pred_rf_2)`: Saya menggunakan metode `accuracy_score` dari scikit-learn untuk menghitung akurasi model Random Forest yang menggunakan ekstraksi fitur TF-IDF. Akurasi adalah metrik yang mengukur sejauh mana model berhasil dalam melakukan prediksi yang benar. Saya membandingkan label sebenarnya dalam himpunan data uji (`y_test`) dengan hasil prediksi yang dihasilkan oleh model Random Forest (`y_pred_rf_2`). Hasil akurasi disimpan dalam variabel `accuracy_rf_2`.

2. `confusion_rf_2 = confusion_matrix(y_test, y_pred_rf_2)`: Saya menggunakan metode `confusion_matrix` dari scikit-learn untuk menghitung matriks kebingungan (confusion matrix) untuk model Random Forest dengan ekstraksi fitur TF-IDF. Matriks kebingungan adalah tabel yang memungkinkan saya untuk melihat seberapa baik model mengklasifikasikan data dalam berbagai kategori. Hasil matriks kebingungan disimpan dalam variabel `confusion_rf_2`.

3. `report_rf_2 = classification_report(y_test, y_pred_rf_2, target_names=['Kesehatan', 'Politik'])`: Saya menggunakan metode `classification_report` dari scikit-learn untuk membuat laporan klasifikasi yang mencakup sejumlah metrik evaluasi, seperti presisi, recall, f1-score, dan lainnya. Laporan klasifikasi ini disusun berdasarkan perbandingan antara label sebenarnya dalam himpunan data uji (`y_test`) dengan hasil prediksi model Random Forest (`y_pred_rf_2`). Saya juga menggunakan `target_names` untuk memberikan nama kategori yang sesuai dengan data Anda, yaitu 'Kesehatan' dan 'Politik'.

4. `print("Confusion Matrix - Random Forest:")` dan `print(confusion_rf_2)`: Saya mencetak matriks kebingungan ke layar agar saya bisa melihat bagaimana model Random Forest dengan ekstraksi fitur TF-IDF melakukan klasifikasi pada setiap kategori. Matriks kebingungan akan menampilkan berapa banyak prediksi yang benar dan berapa banyak prediksi yang salah untuk masing-masing kategori.

5. `print("\nClassification Report - Random Forest:")` dan `print(report_rf_2)`: Saya mencetak laporan klasifikasi ke layar. Laporan klasifikasi memberikan wawasan lebih mendalam tentang kinerja model, termasuk presisi, recall, f1-score, dan lainnya, untuk masing-masing kategori yang telah ditentukan.

6. `print("\nNilai Akurasi - Random Forest:")` dan `print(accuracy_rf_2)`: Saya mencetak akurasi model ke layar. Akurasi adalah metrik yang mencerminkan sejauh mana model berhasil dalam melakukan klasifikasi dengan benar.

Dengan evaluasi ini, saya dapat memahami sejauh mana model Random Forest yang menggunakan ekstraksi fitur TF-IDF berhasil dalam melakukan klasifikasi pada data uji dan apakah perlu dilakukan peningkatan atau penyesuaian pada model tersebut. Evaluasi ini memberikan gambaran tentang kinerja model dalam membedakan antara kategori 'Kesehatan' dan 'Politik' dengan ekstraksi fitur Term Frequency-Inverse Document Frequency (TF-IDF).

Hasil evaluasi model Random Forest dengan ekstraksi fitur TF-IDF adalah sebagai berikut:

**Confusion Matrix - Random Forest**:
- Terdapat dua kategori yang dievaluasi, yaitu "Kesehatan" dan "Politik."
- Untuk kategori "Kesehatan," model berhasil mengklasifikasikan 1 sampel dengan benar, dan tidak ada prediksi yang salah (0).
- Untuk kategori "Politik," model berhasil mengklasifikasikan 2 sampel dengan benar, dan tidak ada prediksi yang salah (0).

**Classification Report - Random Forest**:
- **Presisi (precision)**, **recall**, dan **F1-score** untuk kedua kategori ("Kesehatan" dan "Politik") adalah 1.00. Ini mengindikasikan bahwa model berhasil dalam memprediksi dengan sempurna untuk kedua kategori.
- Jumlah sampel dalam kategori "Kesehatan" adalah 1, sedangkan dalam kategori "Politik" adalah 2.
- **Akurasi (accuracy)** adalah 1.0 (100%), yang berarti model ini berhasil memprediksi dengan benar semua sampel dalam himpunan data uji.

Hasil ini menunjukkan bahwa model Random Forest dengan ekstraksi fitur TF-IDF telah melakukan prediksi dengan sempurna untuk kedua kategori yang dievaluasi. Semua sampel dalam himpunan data uji telah diklasifikasikan dengan benar.



## 8. Prediksi Model


Prediksi model adalah proses menggunakan model mesin pembelajaran (machine learning) yang telah dilatih sebelumnya untuk menghasilkan perkiraan atau estimasi mengenai data yang belum dilihat sebelumnya. Dalam konteks klasifikasi atau regresi, prediksi model melibatkan pemberian label atau nilai target kepada data baru berdasarkan pemahaman yang diperoleh oleh model dari data pelatihan. Dengan kata lain, model digunakan untuk mengklasifikasikan atau memprediksi data baru berdasarkan pola atau hubungan yang telah dipelajari selama pelatihan.

Proses prediksi model melibatkan langkah-langkah berikut:
1. Input Data Baru: Data baru yang ingin diprediksi harus disiapkan dan diproses sedemikian rupa agar sesuai dengan format yang digunakan selama pelatihan model.

2. Penggunaan Model: Model machine learning yang telah dilatih sebelumnya digunakan untuk melakukan prediksi. Model ini menerima data baru sebagai input.

3. Output Prediksi: Model menghasilkan prediksi atau estimasi berdasarkan data baru yang diberikan. Ini bisa berupa label kategori (klasifikasi) atau nilai numerik (regresi).

Prediksi model dapat digunakan dalam berbagai aplikasi, termasuk pengenalan wajah, klasifikasi teks, diagnosis medis, rekomendasi produk, dan banyak lagi. Keakuratan prediksi model sangat bergantung pada kualitas data pelatihan, pemilihan model yang sesuai, dan parameter yang dioptimalkan selama pelatihan. Prediksi model adalah salah satu tahap penting dalam implementasi solusi berbasis machine learning untuk berbagai masalah.

In [ ]:
# Contoh berita baru
berita_baru = "Majelis Kehormatan Mahkamah Konstitusi (MKMK) memutuskan memberhentikan Ketua MK Anwar Usman."

# 1. Bersihkan dan proses teks berita baru
berita_baru = clean_text(berita_baru)  # Gunakan fungsi clean_text yang telah Anda definisikan

# 2. Gunakan vectorizer yang sama
tfidf_berita_baru = tfidf_vectorizer.transform([berita_baru])  # Menggunakan vectorizer TF-IDF yang sudah ada

# 3. Lakukan prediksi
prediksi = rf_model_2.predict(tfidf_berita_baru) # Sesuaikan mau menggunakan model apa "rf_model_2.predict"

print("Prediksi:", prediksi)


Prediksi: ['Politik']


Kode di atas adalah untuk melakukan prediksi label kategori dari sebuah berita baru menggunakan model Random Forest yang telah dilatih sebelumnya dengan ekstraksi fitur TF-IDF. Berikut penjelasan lengkapnya:

1. `berita_baru = "Majelis Kehormatan Mahkamah Konstitusi (MKMK) memutuskan memberhentikan Ketua MK Anwar Usman."`: Saya mendefinisikan teks berita baru dalam variabel `berita_baru`.

2. `berita_baru = clean_text(berita_baru)`: Saya membersihkan dan memproses teks berita baru dengan memanggil fungsi `clean_text(berita_baru)`. Fungsi `clean_text` digunakan untuk membersihkan teks, menghilangkan karakter khusus, mengubah huruf menjadi huruf kecil, dan melakukan pemrosesan teks lainnya agar sesuai dengan teks yang telah digunakan untuk melatih model.

3. `tfidf_berita_baru = tfidf_vectorizer.transform([berita_baru])`: Saya menggunakan vektorisasi TF-IDF yang telah ada (`tfidf_vectorizer`) untuk mengubah teks berita baru (`berita_baru`) menjadi representasi TF-IDF. Ini penting agar berita baru dapat digunakan dengan model yang telah dilatih dengan ekstraksi fitur TF-IDF yang sama.

4. `prediksi = rf_model_2.predict(tfidf_berita_baru)`: Saya menggunakan model Random Forest yang telah dilatih sebelumnya (`rf_model_2`) untuk melakukan prediksi terhadap berita baru yang telah direpresentasikan dalam bentuk TF-IDF (`tfidf_berita_baru`). Hasil prediksi disimpan dalam variabel `prediksi`.

5. `print("Prediksi:", prediksi)`: Saya mencetak hasil prediksi ke layar. Hasil prediksi adalah label kategori yang diberikan oleh model untuk berita baru tersebut.

Hasil dari kode tersebut adalah sebagai berikut:
```
Prediksi: ['Politik']
```

Hasil ini menunjukkan bahwa model Random Forest dengan ekstraksi fitur TF-IDF memprediksi bahwa berita baru tersebut termasuk dalam kategori "Politik." Ini adalah hasil dari klasifikasi teks berita baru menggunakan model yang telah dilatih sebelumnya. Model ini mengklasifikasikan berita baru ke dalam salah satu dari kategori yang telah diajarkan selama pelatihan.